In [1]:
import numpy as np
# import seaborn as sns
import pandas as pd
import os.path

import matplotlib.pyplot as plt

import tifffile 
import czifile

from skimage import transform
from scipy import ndimage

import random 
import math

In [2]:
patch_size = 32
half_ps = 16
half_half_ps = 8
double_ps = 64
double_double_ps = 128

major_ch = 1


In [3]:
def rotate_coor(x_i,y_i,x_c,y_c,rotate_angle):
 
    rotate_angle = rotate_angle*np.pi/180
 
    x_o = (x_i-x_c)*math.cos(rotate_angle) - (2*y_c-y_i-y_c)*math.sin(rotate_angle) +x_c
    y_o = -(x_i-x_c)*math.sin(rotate_angle) - (2*y_c-y_i-y_c)*math.cos(rotate_angle) +(2*y_c-y_c)

    return([x_o,y_o])

In [4]:
image_folder = '/mnt/d/lding/FA/data/FA_ML_Annabel_20250217/031125/Control'
cell_mask_folder = '/mnt/d/lding/FA/analysis_results/FA_ML_Annabel_20250217/031125/ctrl_ch1_major/code_org_20250422_seg/mask'

In [5]:
from datetime import datetime
now = datetime.now()

time_str = now.strftime("%Y%m%d-%H%M")

In [6]:
time_str

'20250507-1001'

In [7]:


movie_partitioned_data_dir = '/mnt/d/lding/FA/analysis_results/FA_ML_Annabel_20250217/031125/ctrl_ch1_major/patches32_40p_'+time_str
movie_plot_dir = '/mnt/d/lding/FA/analysis_results/FA_ML_Annabel_20250217/031125/ctrl_ch1_major/patches_plot32_40p_'+time_str
os.makedirs(movie_partitioned_data_dir,exist_ok=True)
os.makedirs(movie_plot_dir,exist_ok=True)


In [8]:
data_prep_record = pd.DataFrame(columns={'image_folder','filename','filenameID','x_c','y_c','rand_angle','rand_tx','rand_ty',
                            'x_corner1','x_corner2','x_corner3','x_corner4','y_corner1','y_corner2','y_corner3','y_corner4',
                            'movie_partitioned_data_dir','crop_img_filename','movie_plot_dir','plot_filename'})

In [9]:
filenames = [x for x in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, x)) and ('.czi' in x)]

for filenameID in range(7,12): # len(filenames)):
# for filenameID in range(0,1):
    filename = filenames[filenameID]
    train_img = czifile.imread(os.path.join(image_folder, filename)).squeeze()[major_ch,:,:].astype(float)/255/255
    train_seg = tifffile.imread(os.path.join(cell_mask_folder, "cell_mask_"+filename+".tif")).squeeze().astype(float)

    fig_accu, ax_accu = plt.subplots(1,2, figsize=(15.6,7.8), dpi=256, facecolor='w', edgecolor='k')
    ax_accu[0].imshow(train_img, cmap=plt.cm.gray,vmax=1,vmin=0)
    ax_accu[1].imshow(train_seg, cmap=plt.cm.gray,vmax=1,vmin=0)    
    
    x_num = int(np.floor(train_img.shape[1]/patch_size))
    y_num = int(np.floor(train_img.shape[0]/patch_size))
    
    x_0 = int((train_img.shape[1] - x_num*patch_size)/2)
    y_0 = int((train_img.shape[0] - y_num*patch_size)/2)
    
            
    for x_i in range(x_num):
        for y_i in range(y_num):

            y_c = int(y_0+(y_i-0.5)*patch_size)
            x_c = int(x_0+(x_i-0.5)*patch_size)

            y_left = y_c - patch_size
            x_left = x_c - patch_size

            y_right = y_c + patch_size
            x_right = x_c + patch_size


            if y_left < 0 or x_left < 0 or y_right >= train_img.shape[0] or x_right >= train_img.shape[1]:
                continue

            patch_img = train_img[y_left:y_right,x_left:x_right]
            patch_seg = train_seg[y_left:y_right,x_left:x_right]
            rand_angle = random.random() * 360
            rand_tx = int(random.random() * half_ps - half_half_ps)
            rand_ty = int(random.random() * half_ps - half_half_ps)
            

            rotated_patch_img = transform.rotate(patch_img, rand_angle, resize=False, center=None, order=None, mode='constant', cval=0, clip=True)
            rotated_patch_seg = transform.rotate(patch_seg, rand_angle, resize=False, center=None, order=None, mode='constant', cval=0, clip=True)
                            
            cx_left = half_ps-rand_tx
            cx_right = patch_size+half_ps-rand_tx
            cy_up = half_ps-rand_ty
            cy_down = patch_size+half_ps-rand_ty

            crop_patch_img = rotated_patch_img[half_ps-rand_ty:patch_size+half_ps-rand_ty,half_ps-rand_tx:patch_size+half_ps-rand_tx]
            crop_patch_seg = rotated_patch_seg[half_ps-rand_ty:patch_size+half_ps-rand_ty,half_ps-rand_tx:patch_size+half_ps-rand_tx]

            if((crop_patch_seg.mean())<0.4):
                continue

            crop_img_filename = 'c'+str(filenameID).zfill(4)+'x'+str(x_c).zfill(4)+'y'+str(y_c).zfill(4)+'ps'+str(patch_size)+'.tif'
            tifffile.imsave(os.path.join(movie_partitioned_data_dir,crop_img_filename), crop_patch_img.astype(float)                                        )


            X_TransRotCrop = np.array([cx_left, cx_left, cx_right, cx_right, cx_left])
            Y_TransRotCrop = np.array([cy_up, cy_down, cy_down, cy_up, cy_up])

            [X_bigger_patch, Y_bigger_patch] = rotate_coor(X_TransRotCrop,Y_TransRotCrop,patch_size,patch_size,-rand_angle)


            fig, ax = plt.subplots(2,4, figsize=(15.6,7.8), dpi=256, facecolor='w', edgecolor='k')
            ax[0,0].imshow(train_img, cmap=plt.cm.gray,vmax=1,vmin=0)
            ax[0,0].plot([x_left, x_left, x_right, x_right, x_left],[y_left,y_right, y_right, y_left,y_left],color='r')
            ax[0,0].plot(X_bigger_patch+x_left, Y_bigger_patch+y_left,color='green')

            ax[0,1].imshow(train_seg, cmap=plt.cm.gray,vmax=1,vmin=0)
            ax[0,1].plot([x_left, x_left, x_right, x_right, x_left],[y_left,y_right, y_right, y_left,y_left],color='r')
            ax[0,0].plot(X_bigger_patch+x_left, Y_bigger_patch+y_left,color='green')
            

            ax[0,2].imshow(patch_img, cmap=plt.cm.gray,vmax=1,vmin=0)
            ax[0,2].plot(X_bigger_patch, Y_bigger_patch,color='green')

            ax[0,3].imshow(patch_seg, cmap=plt.cm.gray,vmax=1,vmin=0)
            ax[0,3].plot(X_bigger_patch, Y_bigger_patch,color='green')

            ax[1,0].imshow(rotated_patch_img, cmap=plt.cm.gray,vmax=1,vmin=0)
            ax[1,0].plot([half_ps-rand_tx, half_ps-rand_tx, patch_size+half_ps-rand_tx, patch_size+half_ps-rand_tx, half_ps-rand_tx],
                         [half_ps-rand_ty,patch_size+half_ps-rand_ty, patch_size+half_ps-rand_ty, half_ps-rand_ty,half_ps-rand_ty],color='r')

            ax[1,1].imshow(rotated_patch_seg, cmap=plt.cm.gray,vmax=1,vmin=0)
            ax[1,1].plot([half_ps-rand_tx, half_ps-rand_tx, patch_size+half_ps-rand_tx, patch_size+half_ps-rand_tx, half_ps-rand_tx],
                         [half_ps-rand_ty,patch_size+half_ps-rand_ty, patch_size+half_ps-rand_ty, half_ps-rand_ty,half_ps-rand_ty],color='r')

            ax[1,2].imshow(crop_patch_img, cmap=plt.cm.gray,vmax=1,vmin=0)
            ax[1,3].imshow(crop_patch_seg, cmap=plt.cm.gray,vmax=1,vmin=0)
            
            plot_filename = 'plot_grid_t'+str(filenameID).zfill(4)+'_xc'+str(x_c)+'_yc'+str(y_c)+ '.png'
            fig.savefig(os.path.join(movie_plot_dir,plot_filename),bbox_inch='tight')
            plt.close(fig) 
            
            ax_accu[0].plot(X_bigger_patch+x_left, Y_bigger_patch+y_left,color='green')
            ax_accu[1].plot(X_bigger_patch+x_left, Y_bigger_patch+y_left,color='green')
            
            s = pd.Series([image_folder, filename, filenameID, x_c,y_c,rand_angle,rand_tx,rand_ty,
                X_bigger_patch[0]+x_left,X_bigger_patch[1]+x_left,X_bigger_patch[2]+x_left,X_bigger_patch[3]+x_left,
                Y_bigger_patch[0]+y_left,Y_bigger_patch[1]+y_left,Y_bigger_patch[2]+y_left,Y_bigger_patch[3]+y_left,
                movie_partitioned_data_dir,crop_img_filename,movie_plot_dir,plot_filename],
                index=['image_folder','filename','filenameID','x_c','y_c','rand_angle','rand_tx','rand_ty',
                                        'x_corner1','x_corner2','x_corner3','x_corner4','y_corner1','y_corner2','y_corner3','y_corner4',
                                        'movie_partitioned_data_dir','crop_img_filename','movie_plot_dir','plot_filename'])
            
            data_prep_record = data_prep_record.append(s,ignore_index=True)

    data_prep_record.to_csv(os.path.join(movie_plot_dir,'data_prep_record_'+str(filenameID)+'_t'+str(filenameID)+'.csv'))

    fig_accu.savefig(os.path.join(movie_plot_dir,'grid_t'+str(filenameID).zfill(4)+'.png'))   
    plt.close(fig_accu)   
            

/home/ldin/anaconda3/envs/segmentation/lib/python3.7/site-packages/ipykernel_launcher.py:95: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "bbox_inch" which is no longer supported as of 3.3 and will become an error in 3.6
